In [1]:
# Random Forest Model
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
stats_df = pd.read_csv('stats_target_clean2.csv')
stats_df.head()

,Position,Season,Tm,Team_Name,Lg,G,W,L,W/L%,MOV,...,eFG%,TS%,TOV%,ORB%,FT/FGA,opp_eFG%,opp_TS%,opp_TOV%,opp_ORB%,opp_FT/FGA
0,0,2020-21,WAS,Washington Wizards,NBA,72,34,38,0.472,-1.83,...,0.531,0.569,12.3,21.3,0.221,0.539,0.576,12.5,22.4,0.217
1,0,2020-21,UTA,Utah Jazz,NBA,72,52,20,0.722,9.25,...,0.563,0.597,12.7,24.5,0.195,0.507,0.537,10.3,20.7,0.159
2,0,2020-21,TOR,Toronto Raptors,NBA,72,27,45,0.375,-0.47,...,0.529,0.567,11.9,20.8,0.196,0.543,0.584,14.4,23.7,0.234
3,0,2020-21,SAS,San Antonio Spurs,NBA,72,33,39,0.458,-1.74,...,0.517,0.554,10.2,20.0,0.192,0.541,0.570,11.8,22.7,0.174
4,0,2020-21,SAC,Sacramento Kings,NBA,72,31,41,0.431,-3.68,...,0.549,0.578,12.0,21.3,0.185,0.557,0.591,12.1,25.0,0.199


In [2]:
stats_df.drop('Lg', axis=1, inplace=True)

In [3]:
stats_df.drop(['Season','Tm', 'Team_Name'], axis=1, inplace=True)

In [4]:
stats_df.head()

,Position,G,W,L,W/L%,MOV,SOS,SRS,Pace,ORtg,...,eFG%,TS%,TOV%,ORB%,FT/FGA,opp_eFG%,opp_TS%,opp_TOV%,opp_ORB%,opp_FT/FGA
0,0,72,34,38,0.472,-1.83,-0.01,-1.85,104.1,111.2,...,0.531,0.569,12.3,21.3,0.221,0.539,0.576,12.5,22.4,0.217
1,0,72,52,20,0.722,9.25,-0.29,8.97,98.5,117.6,...,0.563,0.597,12.7,24.5,0.195,0.507,0.537,10.3,20.7,0.159
2,0,72,27,45,0.375,-0.47,-0.07,-0.54,99.2,112.0,...,0.529,0.567,11.9,20.8,0.196,0.543,0.584,14.4,23.7,0.234
3,0,72,33,39,0.458,-1.74,0.15,-1.58,98.9,111.0,...,0.517,0.554,10.2,20.0,0.192,0.541,0.570,11.8,22.7,0.174
4,0,72,31,41,0.431,-3.68,0.23,-3.45,100.0,113.6,...,0.549,0.578,12.0,21.3,0.185,0.557,0.591,12.1,25.0,0.199


In [5]:
# Remove Position target from features data
y = stats_df.Position
X = stats_df.drop(columns=["Position"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.970


In [7]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
22/22 [==============================] - 0s 807us/step - loss: 0.4861 - accuracy: 0.9657
Epoch 2/100
22/22 [==============================] - 0s 760us/step - loss: 0.4177 - accuracy: 0.9657
Epoch 3/100
22/22 [==============================] - 0s 712us/step - loss: 0.3534 - accuracy: 0.9657
Epoch 4/100
22/22 [==============================] - 0s 856us/step - loss: 0.3015 - accuracy: 0.9657
Epoch 5/100
22/22 [==============================] - 0s 760us/step - loss: 0.2619 - accuracy: 0.9657
Epoch 6/100
22/22 [==============================] - 0s 855us/step - loss: 0.2340 - accuracy: 0.9657
Epoch 7/100
22/22 [==============================] - 0s 807us/step - loss: 0.2142 - accuracy: 0.9657
Epoch 8/100
22/22 [==============================] - 0s 664us/step - loss: 0.1995 - accuracy: 0.9657
Epoch 9/100
22/22 [==============================] - 0s 808us/step - loss: 0.1881 - accuracy: 0.9657
Epoch 10/100
22/22 [==============================] - 0s 997us/step - loss: 0.1795 - accura

22/22 [==============================] - 0s 712us/step - loss: 0.0767 - accuracy: 0.9657
Epoch 82/100
22/22 [==============================] - 0s 665us/step - loss: 0.0767 - accuracy: 0.9657
Epoch 83/100
22/22 [==============================] - 0s 807us/step - loss: 0.0759 - accuracy: 0.9657
Epoch 84/100
22/22 [==============================] - 0s 760us/step - loss: 0.0755 - accuracy: 0.9657
Epoch 85/100
22/22 [==============================] - 0s 712us/step - loss: 0.0754 - accuracy: 0.9657
Epoch 86/100
22/22 [==============================] - 0s 665us/step - loss: 0.0751 - accuracy: 0.9657
Epoch 87/100
22/22 [==============================] - 0s 665us/step - loss: 0.0749 - accuracy: 0.9657
Epoch 88/100
22/22 [==============================] - 0s 665us/step - loss: 0.0744 - accuracy: 0.9657
Epoch 89/100
22/22 [==============================] - 0s 760us/step - loss: 0.0739 - accuracy: 0.9657
Epoch 90/100
22/22 [==============================] - 0s 619us/step - loss: 0.0738 - accuracy: 